# Line RT: 1D homogeneous LTE, all constant

## 0) Setup

Define Magritte folders

In [1]:
MagritteSetupFolder = '/home/frederik/Dropbox/Astro/Magritte/modules/setup/'
ProjectFolder       = '/home/frederik/MagritteProjects/Lines_1D_LTE/'

Import tools and libraries

In [2]:
import numpy as np

from bokeh.plotting import figure, show, gridplot
from bokeh.palettes import cividis
from bokeh.io       import output_notebook
output_notebook()

from os   import getcwd
from glob import glob
from sys  import path
path.insert(0, MagritteSetupFolder)

# Import from MagritteSetupFolder
from lineData import LineData, planck, relativeDifference
from setup    import setupMagritte
from model    import model

Loading BokehJS ...

## 1) Define model

Assume a 1D model (i.e. 1 ray) with constant temperature, density and abundances and a linear velocity field, yielding a constant velocity gradient

\begin{align}
T(x)     \ &= \ cte, \\
\rho(x)  \ &= \ cte, \\
n_{i}(x) \ &= \ cte, \\
v(x)     \ &= \ cte.
\end{align}

There is CMB radiation incoming on both ends of the ray as boundary condition.

In [3]:
model = model (dim=1)

model.ncells = 1000

dens = 1.0E+12   # [m^-3]
abun = 1.0E+4    # [m^-3]
temp = 1.0E+02   # [K]
dx   = 1.0E5     # [m]
dT   = 0.0E-1    # [K]
dv   = 0.0E2     # [m/s]

model.density     = [dens     for _ in range(model.ncells)]
model.abundance   = [abun     for _ in range(model.ncells)]
model.temperature = [temp     for _ in range(model.ncells)]

model.x = [dx*i  for i in range(model.ncells)]
model.y = [0.0   for _ in range(model.ncells)]
model.z = [0.0   for _ in range(model.ncells)]

model.vx = [0.0  for _ in range(model.ncells)]
model.vy = [0.0  for _ in range(model.ncells)]
model.vz = [0.0  for _ in range(model.ncells)]

model.boundary = [0, model.ncells-1]

model.defineRays (nsides=1)

model.getNeighborLists ()

# Write new model data
model.writeInput (ProjectFolder + 'Magritte_files/')

# Run setup
setupMagritte (projectFolder = ProjectFolder, runName = '')

'/home/frederik/MagritteProjects/Lines_1D_LTE/io/18-10-29_11:35:45/'

In [4]:
# Plot model
plot_model_1 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_1.line (model.x, model.density)
plot_model_1.xaxis.axis_label = "x [m]"
plot_model_1.yaxis.axis_label = "density [m^-3]"

plot_model_2 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_2.line (model.x, model.abundance)
plot_model_2.xaxis.axis_label = "x [m]"
plot_model_2.yaxis.axis_label = "abundance [xm^-3]"

plot_model_3 = figure(plot_width=400, plot_height=250)
plot_model_3.line (model.x, model.temperature)
plot_model_3.xaxis.axis_label = "x [m]"
plot_model_3.yaxis.axis_label = "temperature [K]"

plot_model_4 = figure(plot_width=400, plot_height=250)
plot_model_4.line (model.x, model.vx)
plot_model_4.xaxis.axis_label = "x [m]"
plot_model_4.yaxis.axis_label = "velocity_x [m^-3]"

plot_model = gridplot ([[plot_model_1, plot_model_2],[plot_model_3, plot_model_4]])

show (plot_model)

### Get Magritte output

In [5]:
ioFolders = glob(ProjectFolder + 'io/*/')
ioFolders.sort()

inputFolders  = [folder +  'input/' for folder in ioFolders]
outputFolders = [folder + 'output/' for folder in ioFolders]

lastOutput = outputFolders[-1]
lastInput  =  inputFolders[-1]

pops_files = glob(lastOutput + 'populations_0*.txt')
Jeff_files = glob(lastOutput + 'Jeff_0*.txt')
J_files    = glob(lastOutput + 'J_*.txt')
nu_files   = glob(lastOutput + 'frequencies_nu*.txt')
lnr_files  = glob(lastOutput + 'frequencies_line_nr*.txt')
eta_files  = glob(lastOutput + 'eta_0*.txt')
chi_files  = glob(lastOutput + 'chi_0*.txt')

pops_files.sort()
Jeff_files.sort()
eta_files.sort()
chi_files.sort()

pops_data = [np.loadtxt(fileName) for fileName in pops_files]
Jeff_data = [np.loadtxt(fileName) for fileName in Jeff_files]
J_data    = [np.loadtxt(fileName) for fileName in J_files]
nu_data   = [np.loadtxt(fileName) for fileName in nu_files]
lnr_data  = [np.loadtxt(fileName) for fileName in lnr_files]
eta_data  = [np.loadtxt(fileName) for fileName in eta_files]
chi_data  = [np.loadtxt(fileName) for fileName in chi_files]

# Import linedata
lineData = LineData (ProjectFolder + '/Magritte_files/linedata/hco+.dat')

In [6]:
for file in Jeff_files:
    print(file)

/home/frederik/MagritteProjects/Lines_1D_LTE/io/18-10-29_11:35:45/output/Jeff_0_0.txt
/home/frederik/MagritteProjects/Lines_1D_LTE/io/18-10-29_11:35:45/output/Jeff_0_1.txt
/home/frederik/MagritteProjects/Lines_1D_LTE/io/18-10-29_11:35:45/output/Jeff_0_final.txt


### Plot output

In [7]:
s_min  = 0
s_max  = model.ncells
s_step = 100
ns     = int((s_max - s_min) / s_step)

In [8]:
plot_pops = figure (title='Level populations', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    plot_pops.line(range(lineData.nlev), pops_data[0][s], color=cividis(ns)[int(s/s_step)], legend=str(s))
plot_pops.xaxis.axis_label = "number of the level"
plot_pops.yaxis.axis_label = "population [m^-3]"
show (plot_pops)

In [9]:
plot_Jeff = figure (title='Total mean intensity', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    plot_Jeff.line(range(lineData.nrad), Jeff_data[1][s], color=cividis(ns)[int(s/s_step)], legend=str(s))
plot_Jeff.xaxis.axis_label = "number of the transition"
plot_Jeff.yaxis.axis_label = "mean intensity J [m^-3]"
show (plot_Jeff)

In [10]:
plot_spectrum = figure (title='Spectrum', width=700, height=500, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    plot_spectrum.line(nu_data[0][s], J_data[0][s], color=cividis(ns)[int(s/s_step)], legend=str(s))
plot_spectrum.xaxis.axis_label = "frequencies [Hz]"
plot_spectrum.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot_spectrum)

## Analytical solution

Assuming a constant source function $S_{\nu}(x)=S_{\nu}$ along the ray and boundary condition $B_{\nu}$ on both sides of the ray, the mean intensity is given by

\begin{align}
    J_{\nu}(\tau(x)) \ = \ S_{\nu} \ + \ \frac{1}{2} \ \left(B_{\nu}-S_{\nu}\right) \ \left[e^{-\tau_{\nu}(x)} + e^{-\tau_{\nu}(L-x)}\right],
\end{align}

where the optical depth $\tau_{\nu}$ is given by

\begin{equation}
    \tau_{\nu}(\ell) \ = \ \int_{0}^{\ell} \text{d} l \ \chi_{\nu}(l) .
\end{equation}

The frequency dependence of the opacity only comes from the line profile

\begin{equation}
    \chi_{\nu}(x) \ = \ \chi_{ij} \phi_{\nu},
\end{equation}

where we assume a Gaussian profile

\begin{equation}
	\phi_{\nu}^{ij}(x) \ = \ \frac{1}{\sqrt{\pi} \ \delta\nu_{ij}} \ \exp \left[-\left(\frac{\nu-\nu_{ij}} {\delta\nu_{ij}(x)}\right)^{2}\right], \hspace{5mm} \text{where} \hspace{5mm} \delta\nu_{ij}(x) \ = \ \frac{\nu_{ij}}{c} \sqrt{ \frac{2 k_{b} T(x)}{m_{\text{spec}}} \ + \ v_{\text{turb}}^{2}(x)}.
\end{equation}

Assuming a constant opacity along the ray, the integral for the optical depth yields

\begin{equation}
  \tau_{\nu}(\ell) \ = \ \chi_{ij} \phi_{\nu} \ell.
\end{equation}

In [11]:
from scipy.special import erf

c     = 2.99792458E+8    # [m/s] speed of light
kb    = 1.38064852E-23   # [J/K] Boltzmann's constant
mp    = 1.6726219E-27    # [kg] proton mass
T_CMB = 2.7254800        # [K] CMB temperature
vturb = 0.12012E3        # [m/s] turbulent speed

inverse_sqrt_pi = 0.5641895835478

pops       = lineData.LTEpop(temp) * abun
emissivity = lineData.lineEmissivity(pops)
opacity    = lineData.lineOpacity(pops)
source     = emissivity / opacity
bcd        = planck(T_CMB, np.array(lineData.frequency))

line = 15

S    =  source[line]
B    =     bcd[line]
chi  = opacity[line]
L    = model.x[-1]
nuij = lineData.frequency[line]
dnu  = nuij/c * np.sqrt(2.0*kb*temp/mp + vturb**2)

def phi(nu):
    return inverse_sqrt_pi / dnu * np.exp(-((nu-nuij)/dnu)**2)

def tau(nu, l):
    return chi * phi(nu) * l

def JJ(tau1, tau2):
    return S + 0.5 * (B-S) * (np.exp(-tau1) + np.exp(-tau2))

def J(nu, x):
    tau1 = tau(nu, x)
    tau2 = tau(nu, L-x)
    return JJ(tau1, tau2)

def relativeError(a,b):
    return 2.0 * np.abs((a-b)/(a+b))

In [12]:
s_min = 0
s_max = model.ncells

plot_line_model = figure(title='Line model: freq dependence', width=450, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    # model
    vv = nuij + 8 * dnu * np.linspace(-1,1,500)
    xx = model.x[s]
    plot_line_model.line(vv, J(vv,xx), color=cividis(ns)[int(s/s_step)])
    # data
    M = int(lnr_data[0][s][line] - 18    )
    N = int(lnr_data[0][s][line] + 18 + 1)
    plot_line_model.circle(nu_data[0][s][M:N], J_data[0][s][M:N], color=cividis(ns)[int(s/s_step)], legend=str(s))

plot_line_error = figure(title='Line error', width=450, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 18    )
    N = int(lnr_data[0][s][line] + 18 + 1)
    vv = nu_data[0][s][M:N]
    xx = model.x[s]
    error = relativeError(J(vv,xx), J_data[0][s][M:N])
    plot_line_error.circle(vv, error, color=cividis(ns)[int(s/s_step)], legend=str(s))

plot_line = gridplot([[plot_line_model, plot_line_error]])

show (plot_line)

In [13]:
plot_line_model = figure(title='Line model', width=450, height=400, y_axis_type="log")
#for s in range(s_min, s_max, 2):
# model
xx = np.array(model.x)
plot_line_model.line(xx, J(nuij,xx))
# data
M = int(lnr_data[0][0][line])
plot_line_model.circle(xx, J_data[0][:,M])

plot_line = gridplot([[plot_line_model, plot_line_error]])

show (plot_line)

In [18]:
def relativeError(a,b):
    return 2.0 * np.abs((a-b)/(a+b))

plot_line_model = figure(title='Error between line model and Magritte', width=700, height=500, y_axis_type="log")
#for s in range(s_min, s_max, 2):
# model
xx = np.array(model.x)
#plot_line_model.line(xx, J(nuij,xx))
# data
M = int(lnr_data[0][0][line])
plot_line_model.circle(xx, relativeError(J_data[0][:,M], J(nuij,xx)))
show(plot_line_model)

In [58]:
def diffOperator (u,dtau):
    result = [0.0 for _ in range(len(u))]
    for i in range(1, len(u)-1):
        result[i] = u[i] - ((u[i+1]-u[i])/dtau[i+1] - (u[i]-u[i-1])/dtau[i]) / ((dtau[i+1] + dtau[i])/2.0)
    return result